In [1]:
import pandas as pd
import numpy as np
import re
import mysql.connector
# https://github.com/google/sentencepiece
import sentencepiece as spm

In [ ]:
db = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="root",
  database="opencopora"
)
sql = db.cursor()

In [ ]:
# список id источников
source_ids = [
    1,
    8,
    56,
    184,
    226,
    806,
    1651,
    1675,
    1724,
    2037,
    3469,
    3477,
    3984,
    3994
];
# получаем все источники документов
sql.execute("SELECT DISTINCT bk.book_id, bk.book_name, bk.parent_id, bk.syntax_on, src.url, urls.`filename` \
FROM `opencopora.ru`.books bk \
LEFT JOIN `opencopora.ru`.`sources` src \
ON src.book_id=bk.book_id \
LEFT JOIN `opencopora.ru`.`downloaded_urls` urls \
ON src.url=urls.url \
WHERE bk.book_id IN (1,8,56,184,226,806,1651,1675,1724,2037,3469,3477,3984,3994)")
 
document_sources = pd.DataFrame.from_records(sql.fetchall(), columns=['book_id', 'book_name', 'parent_id', 'syntax_on', 'url', 'filename'])
document_sources.set_index('book_id')

document_sources.head(20)

In [ ]:
import csv
import os
import wget
import shutil
import xml.dom.minidom

# извлечение всех предложений из документа
def extract_sentences(doc_id):
    # получаем все предложения из документа, запоминая разбиение на абзацы
    sql.execute("SELECT sent.sent_id, sent.par_id, sent.`source`, par.book_id, par.pos as par_pos, sent.pos as sent_pos \
    FROM `opencopora.ru`.sentences sent \
    RIGHT JOIN `opencopora.ru`.paragraphs par \
    ON sent.par_id=par.par_id \
    WHERE par.book_id=%s ORDER BY par_pos, sent_pos", (doc_id,))
    return pd.DataFrame.from_records(sql.fetchall(), columns=['sent_id', 'par_id', 'source', 'book_id', 'par_pos', 'sent_pos'])

# создание текста из предложений документа
def create_text(sentences):
    text = ''
    for index, row in sentences.iterrows():
        text += row['source'] + ' '
    return text

# создаем автоматическое разбиение
def auto_create_sentences(doc_id):
    doc_id    = str(doc_id)
    directory = get_data_dir(doc_id)
    path      = directory + '/' + str(doc_id) + '-text.txt'

# получить путь к папке с файлами одного документа
def get_data_dir(doc_id):
    doc_id    = str(doc_id)
    directory = 'dataset/' + doc_id;
    if not os.path.exists(directory):
        os.makedirs(directory)
    return directory

# получить url сохраненной копии документа
def get_saved_source_url(filename):
    return 'http://opencorpora.org/files/saved/' + str(filename) + '.html'

# сохранение текста, разбитого на предложения вручную
def save_sentences(doc_id):
    doc_id    = str(doc_id)
    directory = get_data_dir(doc_id)
    path      = directory + '/' + doc_id + '-sentences.csv'
    sentences = extract_sentences(doc_id)
    sentences.to_csv(path, index=False, escapechar='\\', quoting=csv.QUOTE_NONNUMERIC)
    
    
# сохранение текста, созданного из предложений
def save_text(doc_id):
    doc_id    = str(doc_id)
    directory = get_data_dir(doc_id)
    path      = directory + '/' + str(doc_id) + '-text.txt'
    text      = create_text(extract_sentences(doc_id))
    with open(path, 'w') as txt_file:
        txt_file.write(text)

# сохранение оригинала документа
def save_original_document(filename, doc_id):
    doc_id       = str(doc_id)
    directory    = get_data_dir(doc_id)
    path         = directory + '/' + doc_id + '-original.html'
    document_url = get_saved_source_url(filename)
    wget.download(document_url, path)
    
# сохранение данных полной ручной разметки документа в xml
def save_annotations(doc_id):
    doc_id    = str(doc_id)
    src_path  = 'opcorpora-documents/' + doc_id + '.xml'
    dest_path = get_data_dir(doc_id) + '/' + doc_id + '-annotations.xml'
    # форматируем xml чтобы его было легче читать
    raw_xml = xml.dom.minidom.parse(src_path)
    with open(dest_path, 'w') as xml_file:
        xml_file.write(raw_xml.toprettyxml())
        
# получение списка документов для каждого источника
def get_source_documents(source_id):
    # получаем все источники документов
    sql.execute("SELECT DISTINCT bk.book_id, bk.book_name, bk.parent_id, bk.syntax_on, src.url, urls.`filename` \
    FROM `opencopora.ru`.books bk \
    RIGHT JOIN `opencopora.ru`.`sources` src \
    ON src.book_id=bk.book_id \
    RIGHT JOIN `opencopora.ru`.`downloaded_urls` urls \
    ON src.url=urls.url \
    WHERE bk.parent_id=%s", (source_id,))
    return pd.DataFrame.from_records(sql.fetchall(), columns=['book_id', 'book_name', 'parent_id', 'syntax_on', 'url', 'filename'])

In [ ]:
for source_id in source_ids:
    source_documents = get_source_documents(source_id)
    for index, doc in source_documents.iterrows():
        if os.path.exists('dataset/' + str(doc['book_id'])):
            continue
        # сохраняем предложения и текст
        try:
            save_sentences(doc['book_id'])
            save_text(doc['book_id'])
        except Exception:
            print("Не удалось произвести разбор предложений документа" + str(doc['book_id']))
            continue
        # сохраняем разметку документа
        try:
            save_annotations(doc['book_id'])
        except Exception:
            print("Не удалось сохранить аннотацию документа " + str(doc['book_id']))
        # сохраняем оригинал документа
        try:
            save_original_document(doc['filename'], doc['book_id'])
        except Exception:
            print("Не удалось сохранить оригинал документа " + str(doc['book_id']))

In [ ]:
# создать обучающую выборку для автоматического разбиения текста на предложения
# создает текстовый файл в который записаны все предложения из всех текстов языкового корпуса
def create_train_set():
    sql.execute("SELECT sent_id, source FROM sentences")
    
    train_sentences = pd.DataFrame.from_records(sql.fetchall(), columns=['sent_id', 'source'])
    
    train_text =''
    for index, row in train_sentences.iterrows():
        train_text += row['source'] + "\n"
        
    with open('train.txt', 'w') as txt_file:
        txt_file.write(train_text)

In [ ]:
# обучение модели
# https://github.com/google/sentencepiece/blob/master/python/README.md
spm.SentencePieceTrainer.Train('--input=train.txt --model_prefix=m --vocab_size=32000')